# OSCON 2019 Demo

## Sections
* Required tools
 * Docker and k8s: download and install [docker desktop](https://www.docker.com/products/docker-desktop)
  * Enable kubernetes in docker desktop (preferences->kubernetes->enable)
 * download and install [redis](https://redis.io/download) (>5.0)
 * download and install [kafka](https://kafka.apache.org/downloads)
* Model
 * Train the model: Use the train model (train_model.ipynb)[notebook]
 * Copy the model to the other pods
 * Deploy model to kubeflow (seldon)
* Run kafka
 * Run producer
  * Deploy consumer to kubernetes
* Run redis (single instance)
 * Run redis-streams consumer
* Build and deploy consumer pods
* Publish some raw features and confirm output
 
(It is assumed that you run the notebook from the oscon_demo directory)

#### Train the model from the [notebook](train_model.ipynb)
Then copy the model to the pods

In [ ]:
%cp sklearn_housing_predictor_model/example_model.pkl invoke_model_direct/
%cp sklearn_housing_predictor_model/example_model.pkl invoke_model_seldon/

In [ ]:
# cleanup
!kill -9 `ps -ef|grep -i kafka|grep oscon_demo|awk '{print $2}'`
!rm -rf kafka_2.11-2.2.0*
!rm -rf /tmp/kafka-logs
!rm -rf /tmp/zookeeper
!docker stop test-redis
!docker rm test-redis

#### Build and deploy the model to seldon

In [ ]:
# Build
%cd sklearn_housing_predictor_model/
!sh build_local_img_model.sh
!sh deploy.sh
%cd ..

#### Test the seldon model

In [ ]:
%cd metrics/
!sh test_local_seldon.sh
%cd ..
%killbgscripts

### Download and extract kafka

In [ ]:
!wget http://mirrors.ocf.berkeley.edu/apache/kafka/2.2.0/kafka_2.11-2.2.0.tgz
!tar zxvf kafka_2.11-2.2.0.tgz

### Run Kafka

In [ ]:
# A kafka docker image here would make it simpler
%cd kafka_2.11-2.2.0
f = open("run_zk_and_kafka.sh", "a")
f.write("#!/bin/bash\n\n")
f.write("bin/zookeeper-server-start.sh config/zookeeper.properties &>zk.out &\n")
f.write("sleep 5\n")
f.write("bin/kafka-server-start.sh config/server.properties &>kafka.out &\n")
f.close()
!chmod 755 run_zk_and_kafka.sh

import subprocess
subprocess.call(['./run_zk_and_kafka.sh'])
%cd ..

### Run redis

In [ ]:
!docker run --rm -it -p 6379:6379 --name test-redis -d redis

### Build and deploy containers

#### Raw feature consumer pod

In [ ]:
%cd raw_kafka_feature_consumer_pod/
!sh ./build_img.sh 0.1
!sh ./deploy.sh
%cd ..

#### Direct model invocation pod

In [ ]:
%cd invoke_model_direct/
!sh ./build_local_img_model.sh 0.1
!sh ./deploy.sh
%cd ..

#### Seldon model invocation pod

In [ ]:
%cd invoke_model_seldon/
!sh ./build_local_img_model.sh 0.1
!sh ./deploy.sh
%cd ..

### Finally, publish a set of raw features

In [ ]:
from kafka import KafkaProducer
import numpy as np
from sklearn import datasets
from time import sleep

# start our producer
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

# fetch and load the housing data set
d = datasets.fetch_california_housing()

# get a random row
random_row = d.data[np.random.randint(len(d.data))]

# for demo purposes, divide each geographical unit by 5 (to aggregate later)
divided_features = np.concatenate((random_row[0:6] / 5, np.array([random_row[6], random_row[7]])), axis=0)
short_feature_names = d.feature_names
features_and_names = np.array([short_feature_names, divided_features])

# publish divided features
for i in range(0, 5):
    future = producer.send('housing-topic', features_and_names.tobytes())
print('produced 5 messages for lat/lon=', np.array([random_row[6], random_row[7]]))
sleep(0.5)

#### Confirm predicts are flowing through the components

For **housing-kafka-consumer** it should look like 

`publishing aggregated features to stream for lat/lon  40.69 / -121.83`

And for **housing-predictor-(direct | seldon)** it should be the same for each, like

`direct msg_id= b'1564016621676-0' , lat/lon= [40.69, -121.83] , prediction = $ 117253.33333333331`

In [ ]:
!kubectl -n kubeflow logs `kubectl -n kubeflow get pods|grep 'housing-kafka-consumer' |awk '{print $1}'`|tail -1

In [ ]:
!kubectl -n kubeflow logs `kubectl -n kubeflow get pods|grep 'housing-predictor-seldon' |awk '{print $1}'`|tail -1

In [ ]:
!kubectl -n kubeflow logs `kubectl -n kubeflow get pods|grep 'housing-predictor-direct' |awk '{print $1}'`|tail -1

#### Running a constant producer

And if you want to run a constant producer and watch the logs, just run `row_kafka_feature_producer/producer.py` manually